In [2]:
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticatation of Kaggle API
api = KaggleApi()
api.authenticate()

# dataset
dataset_path = "animals10"
if not os.path.exists(dataset_path):
    print("Downloading Animals-10 dataset...")
    api.dataset_download_files('alessiocorrado99/animals10', path=dataset_path, unzip=False)

# Extract the dataset
with zipfile.ZipFile(f"{dataset_path}/animals10.zip", 'r') as zip_ref:
    print("Extracting dataset...")
    zip_ref.extractall(dataset_path)

print("Dataset downloaded and extracted successfully.")


Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
Extracting dataset...
Dataset downloaded and extracted successfully.


In [3]:
import shutil
from sklearn.model_selection import train_test_split
import glob


original_data_dir = "animals10/raw-img/"
train_dir = "dataset/training_set/"
test_dir = "dataset/test_set/"

# Create directories
for category in os.listdir(original_data_dir):
    category_path = os.path.join(original_data_dir, category)
    if os.path.isdir(category_path):
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(test_dir, category), exist_ok=True)

        # Split into training and test sets
        images = glob.glob(f"{category_path}/*")
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        # Move files
        for image in train_images:
            shutil.move(image, os.path.join(train_dir, category))
        for image in test_images:
            shutil.move(image, os.path.join(test_dir, category))

print("Dataset organized into training_set and test_set.")


Dataset organized into training_set and test_set.


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Part 1 - Data Preprocessing

# Preprocess training set
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Scale images to into 0 to 1 range
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)

# Preprocess test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Part 2 - Build CNN
cnn = tf.keras.models.Sequential()

# 1st Convolution + Pooling
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

# 2nd Convolution + Pooling
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

# Flatten
cnn.add(tf.keras.layers.Flatten())

# Fully Connected Layers
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))  # Hidden layer
cnn.add(tf.keras.layers.Dense(units=len(train_generator.class_indices), activation='softmax'))  # Output layer

# Compile the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
cnn.fit(x=train_generator, validation_data=test_generator, epochs=25)

# Save the model after training
cnn.save('trained_model.h5')




Found 20938 images belonging to 10 classes.
Found 5241 images belonging to 10 classes.


C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
655/655 ━━━━━━━━━━━━━━━━━━━━ 111s 167ms/step - accuracy: 0.3165 - loss: 1.9228 - val_accuracy: 0.4312 - val_loss: 1.7423
Epoch 2/25
655/655 ━━━━━━━━━━━━━━━━━━━━ 102s 155ms/step - accuracy: 0.5336 - loss: 1.3791 - val_accuracy: 0.5432 - val_loss: 1.3203
Epoch 3/25
655/655 ━━━━━━━━━━━━━━━━━━━━ 108s 164ms/step - accuracy: 0.5830 - loss: 1.2412 - val_accuracy: 0.5825 - val_loss: 1.2330
Epoch 4/25
655/655 ━━━━━━━━━━━━━━━━━━━━ 115s 176ms/step - accuracy: 0.6207 - loss: 1.1261 - val_accuracy: 0.6180 - val_loss: 1.1268
Epoch 5/25
655/655 ━━━━━━━━━━━━━━━━━━━━ 115s 175ms/step - accuracy: 0.6594 - loss: 1.0158 - val_accuracy: 0.5884 - val_loss: 1.2343
Epoch 6/25
655/655 ━━━━━━━━━━━━━━━━━━━━ 120s 183ms/step - accuracy: 0.6718 - loss: 0.9617 - val_accuracy: 0.6012 - val_loss: 1.1813
Epoch 7/25
655/655 ━━━━━━━━━━━━━━━━━━━━ 106s 162ms/step - accuracy: 0.6967 - loss: 0.8986 - val_accuracy: 0.6440 - val_loss: 1.0879
Epoch 8/25
655/655 ━━━━━━━━━━━━━━━━━━━━ 110s 168ms/step - accuracy: 0.7122 -

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/test_set/dog/some_image.jpg'

In [ ]:
# Part 3 - Making Predictions
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load the saved model
cnn = load_model('trained_model.h5')


# Load and preprocess a single image
test_image = image.load_img(r'D:/animal classification/dataset/test_set/farfalla/e832b70e2bf71c22d2524518b7444f92e37fe5d404b0144390f8c770a2e9b5_640.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Make prediction
result = cnn.predict(test_image)
predicted_class = np.argmax(result)
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}  # Reverse mapping
prediction = class_labels[predicted_class]

print(f"Predicted Class: {prediction}")


NameError: name 'cnn' is not defined

In [17]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

# Load test data for evaluation
def load_test_data(test_set):
    X_test, y_test = [], []
    for images, labels in test_set:
        X_test.extend(images)
        y_test.extend(labels)
    return np.array(X_test), np.array(y_test)

def evaluate_model(model, test_set):
    # Load test data
    print("Loading test data...")
    X_test, y_test = load_test_data(test_set)
    print(f"Loaded test data: {X_test.shape} images, {y_test.shape} labels")

    # Normalize test data if required
    X_test = X_test / 255.0

    # Get model predictions
    print("Running predictions...")
    y_pred = model.predict(X_test)
    print(f"Predictions shape: {y_pred.shape}")
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    print("Confusion Matrix:")
    print(cm)

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_classes))

    # Accuracy Score
    accuracy = accuracy_score(y_test, y_pred_classes)
    print(f"\nAccuracy Score: {accuracy * 100:.2f}%")




